In [1]:
!pip install docker
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6 libfontconfig1 libxrender1
!pip install hydro_serving_grpc
!pip install hs
!pip install protobuf==3.4.0

You are using pip version 9.0.1, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Unable to locate an executable at "/Library/Java/JavaVirtualMachines/jdk1.8.0_131.jdk/Contents/Home/bin/apt" (-1)
You are using pip version 9.0.1, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from hydroserving.httpclient import HydroservingClient
hs_api = HydroservingClient("http://localhost:8080")

hs_api.models.list()

[{'lastModelBuild': {'finished': '2018-04-26T18:47:20.468',
   'id': 1,
   'model': {'created': '2018-04-26T18:47:03.876',
    'id': 1,
    'modelContract': 'model_name: ""\nsignatures {\n  signature_name: "detect"\n  inputs {\n    name: "result"\n    shape {\n      dim {\n        size: 1\n        name: ""\n      }\n      dim {\n        size: 4\n        name: ""\n      }\n      unknown_rank: false\n    }\n    dtype: DT_DOUBLE\n  }\n  outputs {\n    name: "result"\n    shape {\n      dim {\n        size: 1\n        name: ""\n      }\n      dim {\n        size: 4\n        name: ""\n      }\n      unknown_rank: false\n    }\n    dtype: DT_DOUBLE\n  }\n  outputs {\n    name: "anomaly"\n    dtype: DT_BOOL\n  }\n}\n',
    'modelType': 'python:3.6',
    'name': 'demo_postprocessing',
    'source': 'local:demo_postprocessing',
    'updated': '2018-04-26T18:47:27.091'},
   'modelVersion': {'created': '2018-04-26T18:47:20.454',
    'id': 1,
    'imageName': 'demo_postprocessing',
    'imageSHA25

In [2]:
py = hs_api.runtimes.create('hydrosphere/serving-runtime-python', '3.6-latest', ["python:3.6"])
tf = hs_api.runtimes.create('hydrosphere/serving-runtime-tensorflow', '1.4.0-latest', ["tensorflow"])
# py = hs_api.runtimes.find('hydrosphere/serving-runtime-python', '3.6-latest')
# tf = hs_api.runtimes.find('hydrosphere/serving-runtime-tensorflow', '1.4.0-latest')
print(py)
print(tf)

{'name': 'hydrosphere/serving-runtime-python', 'tags': [], 'version': '3.6-latest', 'id': 2, 'configParams': {}, 'suitableModelType': ['python:3.6']}
{'name': 'hydrosphere/serving-runtime-tensorflow', 'tags': [], 'version': '1.4.0-latest', 'id': 3, 'configParams': {}, 'suitableModelType': ['tensorflow']}


In [15]:
# build model version
preprocessing_model = list(filter(lambda m: m['model']['name'] == 'demo_preprocessing', hs_api.models.list()))[0]
preprocessing_version = hs_api.models.build(preprocessing_model['model']['id'])

print(preprocessing_version)

{'source': 'local:demo_preprocessing', 'model': {'name': 'demo_preprocessing', 'source': 'local:demo_preprocessing', 'modelContract': 'model_name: ""\nsignatures {\n  signature_name: "normalize"\n  inputs {\n    name: "data"\n    shape {\n      dim {\n        size: 24\n        name: ""\n      }\n      unknown_rank: false\n    }\n    dtype: DT_DOUBLE\n  }\n  outputs {\n    name: "data"\n    shape {\n      dim {\n        size: 24\n        name: ""\n      }\n      unknown_rank: false\n    }\n    dtype: DT_DOUBLE\n  }\n}\n', 'modelType': 'python:3.6', 'id': 3, 'updated': '2018-04-26T15:02:30.537', 'created': '2018-04-26T15:02:30.537'}, 'modelContract': 'model_name: ""\nsignatures {\n  signature_name: "normalize"\n  inputs {\n    name: "data"\n    shape {\n      dim {\n        size: 24\n        name: ""\n      }\n      unknown_rank: false\n    }\n    dtype: DT_DOUBLE\n  }\n  outputs {\n    name: "data"\n    shape {\n      dim {\n        size: 24\n        name: ""\n      }\n      unknown_ran

In [16]:
greenhouse = list(filter(lambda m: m['model']['name'] == 'demo_postprocessing', hs_api.models.list()))[0]
greenhouse_version = hs_api.models.build(greenhouse['model']['id'])
print(greenhouse_version)

{'source': 'local:demo_postprocessing', 'model': {'name': 'demo_postprocessing', 'source': 'local:demo_postprocessing', 'modelContract': 'model_name: ""\nsignatures {\n  signature_name: "detect"\n  inputs {\n    name: "result"\n    shape {\n      dim {\n        size: 4\n        name: ""\n      }\n      unknown_rank: false\n    }\n    dtype: DT_DOUBLE\n  }\n  outputs {\n    name: "result"\n    shape {\n      dim {\n        size: 4\n        name: ""\n      }\n      unknown_rank: false\n    }\n    dtype: DT_DOUBLE\n  }\n  outputs {\n    name: "anomaly"\n    dtype: DT_BOOL\n  }\n}\n', 'modelType': 'python:3.6', 'id': 2, 'updated': '2018-04-26T15:02:29.697', 'created': '2018-04-26T15:02:29.697'}, 'modelContract': 'model_name: ""\nsignatures {\n  signature_name: "detect"\n  inputs {\n    name: "result"\n    shape {\n      dim {\n        size: 4\n        name: ""\n      }\n      unknown_rank: false\n    }\n    dtype: DT_DOUBLE\n  }\n  outputs {\n    name: "result"\n    shape {\n      dim {\n 

In [17]:
# Could fail with timeout, since Tensorflow dependecy is big
lstm = list(filter(lambda m: m['model']['name'] == 'demo_lstm', hs_api.models.list()))[0]
lstm_version = hs_api.models.build(lstm['model']['id'])
print(lstm_version)

{'source': 'local:demo_lstm', 'model': {'name': 'demo_lstm', 'source': 'local:demo_lstm', 'modelContract': 'model_name: ""\nsignatures {\n  signature_name: "infer"\n  inputs {\n    name: "data"\n    shape {\n      dim {\n        size: -1\n        name: ""\n      }\n      dim {\n        size: -1\n        name: ""\n      }\n      dim {\n        size: 24\n        name: ""\n      }\n      unknown_rank: false\n    }\n    dtype: DT_FLOAT\n  }\n  outputs {\n    name: "result"\n    shape {\n      dim {\n        size: 4\n        name: ""\n      }\n      unknown_rank: false\n    }\n    dtype: DT_FLOAT\n  }\n}\n', 'modelType': 'tensorflow', 'id': 1, 'updated': '2018-04-26T15:02:28.527', 'created': '2018-04-26T15:02:28.527'}, 'modelContract': 'model_name: ""\nsignatures {\n  signature_name: "infer"\n  inputs {\n    name: "data"\n    shape {\n      dim {\n        size: -1\n        name: ""\n      }\n      dim {\n        size: -1\n        name: ""\n      }\n      dim {\n        size: 24\n        nam

In [18]:
lstm_version = list(filter(lambda m: m['model']['name'] == 'demo_lstm', hs_api.models.list()))[0]["lastModelVersion"]
print(lstm_version)

{'source': 'local:demo_lstm', 'model': {'name': 'demo_lstm', 'source': 'local:demo_lstm', 'modelContract': 'model_name: ""\nsignatures {\n  signature_name: "infer"\n  inputs {\n    name: "data"\n    shape {\n      dim {\n        size: -1\n        name: ""\n      }\n      dim {\n        size: -1\n        name: ""\n      }\n      dim {\n        size: 24\n        name: ""\n      }\n      unknown_rank: false\n    }\n    dtype: DT_FLOAT\n  }\n  outputs {\n    name: "result"\n    shape {\n      dim {\n        size: 4\n        name: ""\n      }\n      unknown_rank: false\n    }\n    dtype: DT_FLOAT\n  }\n}\n', 'modelType': 'tensorflow', 'id': 1, 'updated': '2018-04-26T15:02:28.527', 'created': '2018-04-26T15:02:28.527'}, 'modelContract': 'model_name: ""\nsignatures {\n  signature_name: "infer"\n  inputs {\n    name: "data"\n    shape {\n      dim {\n        size: -1\n        name: ""\n      }\n      dim {\n        size: -1\n        name: ""\n      }\n      dim {\n        size: 24\n        nam

In [22]:
# create application
app_name = "stateful_lstm_pipeline_2"

application = {
  "name": app_name,
  "executionGraph": {
    "stages": [
      {
        "services": [
          {
            "runtimeId": py["id"],
            "modelVersionId": preprocessing_version["id"],
            "weight": 100,
            "signatureName": "normalize"
          }
        ]
      },
        {
        "services": [
          {
            "runtimeId": tf["id"],
            "modelVersionId": lstm_version["id"],
            "weight": 100,
            "signatureName": "infer"
          }
        ]
      },
        {
        "services": [
          {
            "runtimeId": py["id"],
            "modelVersionId": greenhouse_version["id"],
            "weight": 100,
            "signatureName": "detect"
          }
        ]
      }
    ]
  },
  "kafkaStreaming": []
}

created_app = hs_api.applications.create(application)
print(created_app)

{'name': 'stateful_lstm_pipeline_2', 'kafkaStreaming': [], 'id': 2, 'executionGraph': {'stages': [{'services': [{'serviceDescription': {'runtimeId': 2, 'modelVersionId': 1, 'modelName': 'demo_preprocessing:1', 'runtimeName': 'hydrosphere/serving-runtime-python'}, 'weight': 100, 'signature': 'signature_name: "normalize"\ninputs {\n  name: "data"\n  shape {\n    dim {\n      size: 24\n      name: ""\n    }\n    unknown_rank: false\n  }\n  dtype: DT_DOUBLE\n}\noutputs {\n  name: "data"\n  shape {\n    dim {\n      size: 24\n      name: ""\n    }\n    unknown_rank: false\n  }\n  dtype: DT_DOUBLE\n}\n'}], 'signature': 'signature_name: "0"\ninputs {\n  name: "data"\n  shape {\n    dim {\n      size: 24\n      name: ""\n    }\n    unknown_rank: false\n  }\n  dtype: DT_DOUBLE\n}\noutputs {\n  name: "data"\n  shape {\n    dim {\n      size: 24\n      name: ""\n    }\n    unknown_rank: false\n  }\n  dtype: DT_DOUBLE\n}\n'}, {'services': [{'serviceDescription': {'runtimeId': 3, 'modelVersionId': 

In [24]:
import hydro_serving_grpc as hs
import os
import grpc
import base64
import numpy as np
import random

channel = grpc.insecure_channel('localhost:8080')
client = hs.PredictionServiceStub(channel=channel)

shape = hs.TensorShapeProto(dim=[
    hs.TensorShapeProto.Dim(size=24)
])

data_tensor = hs.TensorProto(
    dtype=hs.DT_DOUBLE,
    tensor_shape=shape,
    double_val=[x for x in np.random.random(24).tolist()]
)

request = hs.PredictRequest(
    model_spec=hs.ModelSpec(signature_name=app_name, name=app_name),
    inputs={
        "data": data_tensor,
    }
)

result = client.Predict(request)
print(result)

_Rendezvous: <_Rendezvous of RPC that terminated with (StatusCode.CANCELLED, Received http2 header with status: 503)>